### SVM Best Model  
- RandomizedSearchCV : params :{'gamma': 0.01, 'C': 0.001})/ roc_auc_score_val : 0.7827199859717844

- GridSearchCV: {'C': 0.001, 'gamma': 0.0001} / roc_auc_score_val:0.8605244741486768


  =>RandomizedSearchCV 에서 찾은 'gamma': 0.01, 'C': 0.001 이 들어 갈 수있는 범위를 만들어 준다 


In [3]:
import pandas as pd 
import numpy as np
data =pd.read_csv("preprocessing\cs_data.csv",
                 header=0,
                 index_col=0)

In [4]:
# NA를 전부 없앤 값 = datap
data.dropna(inplace=True)
data.head()

SeriousDlqin2yrs  RevolvingUtilizationOfUnsecuredLines  age  \
1                 1                              0.766127   45   
2                 0                              0.957151   40   
3                 0                              0.658180   38   
4                 0                              0.233810   30   
5                 0                              0.907239   49   

   NumberOfTime30-59DaysPastDueNotWorse  DebtRatio  MonthlyIncome  \
1                                     2   0.802982         9120.0   
2                                     0   0.121876         2600.0   
3                                     1   0.085113         3042.0   
4                                     0   0.036050         3300.0   
5                                     1   0.024926        63588.0   

   NumberOfOpenCreditLinesAndLoans  NumberOfTimes90DaysLate  \
1                               13                        0   
2                                4                        0   
3                                2                        1   
4                                5                        0   
5                                7                        0   

   NumberRealEstateLoansOrLines  NumberOfTime60-89DaysPastDueNotWorse  \
1                             6                                     0   
2                             0                                     0   
3                             0                                     0   
4                             0                                     0   
5                             1                                     0   

   NumberOfDependents  
1                 2.0  
2                 1.0  
3                 0.0  
4                 0.0  
5                 0.0

In [5]:
data.shape ,# drop na
data.isnull().sum()


SeriousDlqin2yrs                        0
RevolvingUtilizationOfUnsecuredLines    0
age                                     0
NumberOfTime30-59DaysPastDueNotWorse    0
DebtRatio                               0
MonthlyIncome                           0
NumberOfOpenCreditLinesAndLoans         0
NumberOfTimes90DaysLate                 0
NumberRealEstateLoansOrLines            0
NumberOfTime60-89DaysPastDueNotWorse    0
NumberOfDependents                      0
dtype: int64

In [6]:
#Data 로 X,y 구분 
X= data.drop(columns='SeriousDlqin2yrs')
y= data['SeriousDlqin2yrs']
X.shape , y.shape

((120269, 10), (120269,))

In [7]:
# 1.Dataset 구분 TRAIN, VAL, TEST
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split (X, y, test_size =0.2, stratify=y ,random_state=0) 
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train, random_state=0)

In [8]:
# 2.Scaling
from sklearn.preprocessing import StandardScaler, MinMaxScaler
sc= StandardScaler()
X_train_scaled1=sc.fit_transform(X_train)
X_val_scaled1=sc.transform(X_val)
X_test_scaled1=sc.transform(X_test)


In [ ]:
######### SVM best model 찾기 ########

In [7]:
# Best SVM param을 RandomizedSearchCV 로 찾기 
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.metrics import roc_curve, plot_roc_curve, roc_auc_score

svm = SVC(random_state=0, probability =True)
param={
    "C":[0.001,0.01,0.1,1,10],
    "gamma":[0.001,0.01,0.1,1,10]
}

rs=RandomizedSearchCV(svm, param, scoring = 'roc_auc',n_iter=10, cv=4, n_jobs=-1)

In [8]:
rs.fit(X_train_scaled1, y_train)

RandomizedSearchCV(cv=4, estimator=SVC(probability=True, random_state=0),
                   n_jobs=-1,
                   param_distributions={'C': [0.001, 0.01, 0.1, 1, 10],
                                        'gamma': [0.001, 0.01, 0.1, 1, 10]},
                   scoring='roc_auc')

In [9]:
rs.best_score_ , rs.best_params_

(0.764554088651352, {'gamma': 0.01, 'C': 0.001})

In [10]:
model=rs.best_estimator_

In [14]:
roc_auc_score(y_val, model.predict_proba(X_val_scaled1)[:,1])

0.7827199859717844

In [9]:
# Best SVM param을 GridSearchCV 로 찾기 
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import roc_curve, plot_roc_curve, roc_auc_score

svm = SVC(random_state =0 , probability = True)
param={
    "C":[0.0001,0.001,0.01],
    "gamma":[0.0001,0.001,0.01,0.1]
}

gs=GridSearchCV(svm, param, scoring = 'roc_auc',cv=4, n_jobs=-1)

In [10]:
gs.fit(X_train_scaled1, y_train)

GridSearchCV(cv=4, estimator=SVC(probability=True, random_state=0), n_jobs=-1,
             param_grid={'C': [0.0001, 0.001, 0.01],
                         'gamma': [0.0001, 0.001, 0.01, 0.1]},
             scoring='roc_auc')

In [11]:
gs.best_score_ , gs.best_params_

(0.7880522664341514, {'C': 0.001, 'gamma': 0.0001})

In [15]:
model1=gs.best_estimator_
model1

SVC(C=0.001, gamma=0.0001, probability=True, random_state=0)

In [16]:
roc_auc_score(y_val,model1.predict_proba(X_val_scaled1)[:,1])

0.8068564825485639